### **CREACIÓN DE LA BASE DE DATOS EN PostgreSQL**
---

Este notebook está dedicado a la creación de la base de datos en PostgreSQL para el proyecto de telecomunicaciones. A lo largo de este documento, se explicará paso a paso el proceso de conexión, creación de tablas y carga de los datasets.

### **Proceso de Creación de la Base de Datos**

1) **Conexión** a PostgreSQL.
2) **Creación de la base de datos** y definición del esquema.
3) **Carga de los datasets procesados** en tablas separadas.
4) **Definición de relaciones**.


---
#### **Conexión a PostgreSQL**

In [29]:
import psycopg2
from psycopg2 import sql

In [31]:
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="postgres"
)

# Habilitación de autocommit
connection.autocommit = True

cursor = connection.cursor()

cursor.execute("CREATE DATABASE telecomunicaciones;")
print("Base de datos creada exitosamente.")

cursor.close()
connection.close()

Base de datos creada exitosamente.


---
#### **Creación de tablas (DDL) - Dataset Internet**

- **Esquema y Modelo Relacional**


Se crearon tablas específicas para ``periodos``, ``provincias`` y ``localidades``. Esto responde a la necesidad de evitar redundancia y mantener un modelo limpio, donde las entidades que se repiten en múltiples datasets están centralizadas. De esta forma, podemos hacer referencia a ellas desde otras tablas mediante llaves foráneas.

* **Tablas de Apoyo:**

  1. **Provincias** : Incluye los nombres únicos de las provincias.

  2. **Periodos** : Registra los distintos periodos de tiempo (Año - Trimestre - Períodos en formato trimestral: Ene-Mar 2024).

  3. **Localidades** : Almacena las localidades únicas, asociadas a sus provincias.


- **Decisión sobre el uso de llaves foráneas:**

Todas las relaciones entre tablas que comparten información común (como provincias y periodos) se gestionan a través de llaves foráneas. Esto asegura la integridad referencial en todo momento, garantizando que cualquier dato relacionado se almacene de manera consistente.

In [32]:
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

cursor = connection.cursor()

# Habilitamos autocommit
connection.autocommit = True


#Crear tabla 'periodos'
cursor.execute("""
           
CREATE TABLE periodos (
    id_periodo SERIAL PRIMARY KEY,
    anio INT,
    trimestre INT,
    descripcion_periodo VARCHAR(50)
);
               
""")


#Crear tabla 'provincias'
cursor.execute("""
            
CREATE TABLE provincias (
    id_provincia SERIAL PRIMARY KEY,
    nombre_provincia VARCHAR(100) UNIQUE
);

""")


# Crear tabla 'localidades'
cursor.execute("""
                
CREATE TABLE IF NOT EXISTS localidades (
    id_localidad SERIAL PRIMARY KEY,
    id_provincia INT REFERENCES provincias(id_provincia),
    partido VARCHAR(100),
    localidad VARCHAR(100),
    link_indec VARCHAR(100),
    latitud DECIMAL(9, 6), -- precisión de 6 decimales es suficiente para coordenadas geográficas
    longitud DECIMAL(9, 6)
);

               
""")


# Crear tabla 'accesos_por_velocidad_localidades'
cursor.execute("""
            
CREATE TABLE IF NOT EXISTS accesos_por_velocidad_localidades (
    id_velocidad SERIAL PRIMARY KEY,
    id_localidad INT REFERENCES localidades(id_localidad),
    velocidad_mbps DECIMAL(10, 2),
    cantidad_accesos INT
);
               
""")


# Crear tabla 'tecnologias_acceso"
cursor.execute("""
             
CREATE TABLE tecnologias_acceso (
    id_tecnologia SERIAL PRIMARY KEY,
    id_localidad INT REFERENCES localidades(id_localidad),
    adsl INT,
    cablemodem INT,
    dial_up INT,
    fibra_optica INT,
    otros INT,
    satelital INT,
    wimax INT,
    wireless INT,
    total_general INT
);
               
""")


# Crear tabla accesos_velocidades_provincia
cursor.execute("""
           
CREATE TABLE accesos_velocidad_provincia (
    id_acceso SERIAL PRIMARY KEY,
    id_provincia INT REFERENCES provincias(id_provincia),
    id_periodo INT REFERENCES periodos(id_periodo),
    hasta_512_kbps INT,
    entre_512_kbps_1_mbps INT,
    entre_1_mbps_6_mbps INT,
    entre_6_mbps_10_mbps INT,
    entre_10_mbps_20_mbps INT,
    entre_20_mbps_30_mbps INT,
    mas_de_30_mbps INT,
    otros INT,
    total_accesos INT
);
               
""")


# Crear tabla 'penetracion_internet_poblacion'
cursor.execute("""
       
CREATE TABLE penetracion_internet_poblacion (
    id_penetracion_poblacion SERIAL PRIMARY KEY,
    id_provincia INT REFERENCES provincias(id_provincia),
    id_periodo INT REFERENCES periodos(id_periodo),
    accesos_por_100_hab DECIMAL(10, 2)
);
               
""")


# Crear tabla 'penetracion_internet_hogares
cursor.execute(""" 

CREATE TABLE penetracion_internet_hogares (
    id_penetracion_hogares SERIAL PRIMARY KEY,
    id_provincia INT REFERENCES provincias(id_provincia),
    id_periodo INT REFERENCES periodos(id_periodo),
    accesos_por_100_hogares DECIMAL(10, 2)
);
               
""")


# Crear tabla 'ingresos_servicios_internet
cursor.execute("""
          
CREATE TABLE ingresos_servicios_internet (
    id_ingresos_internet  SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo),
    ingresos_miles_pesos DECIMAL(15, 2)
);
               
""")


# Crear tabla 'velocidad_media_provinica'
cursor.execute(""" 

CREATE TABLE velocidad_media_provincia (
    id_velocidad SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo),
    id_provincia INT REFERENCES provincias(id_provincia),
    mbps_media_bajada DECIMAL(10, 2)
);
               
""")


print("Todas las tablas fueron creadas exitosamente.")


cursor.close()
connection.close()

Todas las tablas fueron creadas exitosamente.


---

#### **Ingesta de datos (DML) - Dataset Internet**
- **Transacciones**

Se utilizan transacciones para garantizar que los datos se inserten de manera consistente. En caso de que ocurra un error durante la inserción, se hace un rollback para deshacer cualquier cambio.

- **Normalización de nombres de provincias** :

Se estandarizó el nombre de las provincias, reemplazando "Capital Federal" por "CABA". Además se convirtieron todos los nombres de las provincias a minúsculas en cada dataset, para mantener un consistencia en la Base de Datos.
- **Tratamiento de valores nulos y categorías "Otros"** :

En algunas tablas, se eliminaron filas con valores nulos o categorías genéricas como "Otros" y "Sin Datos", para mantener la consistencia en los datos.

#### **Tabla 'periodos'**

- Para la ingesta de datos a la Tabla periodos se utiliza el dataset ``ingresos_telefonia_movil`` dado que contiene el año 2013 como valor mínimo en comparación con otros datasets que comienzan a partir del año 2014.

In [33]:
import pandas as pd

# Carga del dataset portabilidad.csv
file_path = "datasets\dataset_procesados_proyecto\Telefonia_processed\ingresos_telefonia_movil.csv"
df_telef = pd.read_csv(file_path)

# Filtro columnas necesarias
df_periodos = df_telef[['Año', 'Trimestre', 'Periodo']].drop_duplicates()


print(df_periodos.head())


    Año  Trimestre        Periodo
0  2013          1   Ene-Mar 2013
1  2013          2   Abr-Jun 2013
2  2013          3  Jul-Sept 2013
3  2013          4   Oct-Dic 2013
4  2014          1   Ene-Mar 2013


In [34]:
import psycopg2
from psycopg2 import sql

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# DataFrame filtrado para la tabla `periodos`
df_periodos = df_telef[['Año', 'Trimestre', 'Periodo']].drop_duplicates()

# Transacción para la inserción de datos en `periodos`
try:
    for index, row in df_periodos.iterrows():
        cursor.execute("""
            INSERT INTO periodos (anio, trimestre, descripcion_periodo)
            VALUES (%s, %s, %s)
            ON CONFLICT DO NOTHING;
        """, (row['Año'], row['Trimestre'], row['Periodo']))

    # Confirmación
    connection.commit()
    print("Datos insertados correctamente en la tabla 'periodos'")

except Exception as e:
    # Rollback para deshacer cualquier cambio en caso de error
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'periodos': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'periodos'


#### **Tabla 'provincias'**
-  Se utiliza el dataset ``mapa_conectividad`` para la ingesta de los datos.
- Se aplica una función para modificar los nombres de las provincias a minúsculas.

In [35]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres de provincias
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `mapa_conectividad.csv`
file_path = "datasets/dataset_procesados_proyecto/Mapa_conectividad_processed/mapa_conectividad.csv"
df_mapa_conectividad = pd.read_csv(file_path)

# Normalizar los nombres de provincia en el DataFrame
df_mapa_conectividad['Provincia'] = df_mapa_conectividad['Provincia'].apply(normalizar_nombre)

# Ingesta de los datos en la tabla 'provincias'
try:
    for provincia in df_mapa_conectividad['Provincia'].unique():
        cursor.execute("""
            INSERT INTO provincias (nombre_provincia)
            VALUES (%s)
            ON CONFLICT (nombre_provincia) DO NOTHING;
        """, (provincia,))
    
    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'provincias'.")

except Exception as e:
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'provincias': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'provincias'.


#### **Tabla 'localidades'**

-  Se utiliza el dataset ``mapa_conectividad`` para la ingesta de los datos.
- Se aplica una función para modificar los nombres de las localidades a minúsculas.

In [36]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres a minúsculas
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Cargar dataset mapa_conectividad
file_path = 'datasets/dataset_procesados_proyecto/Mapa_conectividad_processed/mapa_conectividad.csv'
df_mapa_conectividad = pd.read_csv(file_path)

# Filtrar las columnas necesarias y normalizar nombres
df_localidades = df_mapa_conectividad[['Provincia', 'Partido', 'Localidad', 'Link', 'Latitud', 'Longitud']].drop_duplicates()
df_localidades['Provincia'] = df_localidades['Provincia'].apply(normalizar_nombre)
df_localidades['Partido'] = df_localidades['Partido'].apply(normalizar_nombre)
df_localidades['Localidad'] = df_localidades['Localidad'].apply(normalizar_nombre)

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

connection.autocommit = False
cursor = connection.cursor()

# Transacción para la inserción de localidades con latitud y longitud
try:
    for index, row in df_localidades.iterrows():
        # Obtener el id_provincia desde la tabla provincias 
        cursor.execute("""
            SELECT id_provincia FROM provincias WHERE LOWER(nombre_provincia) = %s
        """, (row['Provincia'],))
        id_provincia = cursor.fetchone()

        if id_provincia is None:
            print(f"Advertencia: No se encontró provincia para {row['Provincia']}")
            continue

        # Insertar localidad si no existe
        cursor.execute("""
            INSERT INTO localidades (id_provincia, partido, localidad, link_indec, latitud, longitud)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT DO NOTHING;
        """, (id_provincia[0], row['Partido'], row['Localidad'], row['Link'], row['Latitud'], row['Longitud']))

    # Confirmar transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'localidades' con latitud y longitud")

except Exception as e:
    connection.rollback()
    print(f"Error: {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'localidades' con latitud y longitud


#### **Tabla 'velocidad'**
- Se aplica una función para modificar los nombres de las provincias, localidades y partido a minúsculas.


In [40]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres a minúsculas
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `accesos_velocidad_localidad.csv`
file_path = "datasets/dataset_procesados_proyecto/Internet_processed/accesos_velocidad_localidad.csv"
df_accesos_velocidad = pd.read_csv(file_path)

# Filtros y normalización
df_velocidades = df_accesos_velocidad[['Provincia', 'Partido', 'Localidad', 'Link Indec'] + [col for col in df_accesos_velocidad.columns if 'Mbps' in col]].drop_duplicates()
df_velocidades['Provincia'] = df_velocidades['Provincia'].apply(normalizar_nombre)
df_velocidades['Partido'] = df_velocidades['Partido'].apply(normalizar_nombre)
df_velocidades['Localidad'] = df_velocidades['Localidad'].apply(normalizar_nombre)

# Transacción para la inserción de datos en `accesos_por_velocidad_localidades`
try:
    for index, row in df_velocidades.iterrows():
        # Obtener el id_provincia desde la tabla provincias
        cursor.execute("""
            SELECT id_provincia FROM provincias WHERE LOWER(nombre_provincia) = %s
        """, (row['Provincia'],))
        id_provincia_result = cursor.fetchone()

        if id_provincia_result is not None:
            id_provincia = id_provincia_result[0]  # Obtener el id_provincia correspondiente

            # Obtener el id_localidad desde la tabla localidades 
            cursor.execute("""
                SELECT id_localidad FROM localidades WHERE LOWER(localidad) = %s AND LOWER(partido) = %s AND id_provincia = %s
            """, (row['Localidad'], row['Partido'], id_provincia))
            id_localidad_result = cursor.fetchone()

            if id_localidad_result is not None:
                id_localidad = id_localidad_result[0]  # Obtener el id_localidad correspondiente

                # Iterar sobre cada columna de velocidad
                for col in df_velocidades.columns[4:]:  # Desde la quinta columna en adelante (que son las velocidades)
                    if not pd.isna(row[col]) and row[col] > 0:  # Verificar que el valor no sea nulo o 0
                        # Insertar en la tabla velocidades
                        cursor.execute("""
                            INSERT INTO accesos_por_velocidad_localidades (id_localidad, velocidad_mbps, cantidad_accesos)
                            VALUES (%s, %s, %s)
                        """, (id_localidad, float(col.replace(' Mbps', '').replace(',', '.')), int(row[col])))
            else:
                print(f"Advertencia: No se encontró localidad para {row['Localidad']} en {row['Partido']}, {row['Provincia']}")
        else:
            print(f"Advertencia: No se encontró provincia para {row['Provincia']}")

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'velocidades'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'velocidades': {e}")

finally:
    cursor.close()
    connection.close()


Advertencia: No se encontró localidad para alfredo demarchi  (est. facundo quiroga) en 9 de julio, buenos aires
Advertencia: No se encontró localidad para manuel b. gonnet  (est. french) en 9 de julio, buenos aires
Advertencia: No se encontró localidad para villa general fournier  (est. 9 de julio sud) en 9 de julio, buenos aires
Advertencia: No se encontró localidad para adolfo gonzales chaves  (est. chaves) en adolfo gonzales chaves, buenos aires
Advertencia: No se encontró localidad para alberti   (est. andres vaccarezza) en alberti, buenos aires
Advertencia: No se encontró localidad para villa ortiz  (est. coronel mom) en alberti, buenos aires
Advertencia: No se encontró localidad para otros en amba, buenos aires
Advertencia: No se encontró localidad para general daniel cerri  (est. general cerri) en bahia blanca, buenos aires
Advertencia: No se encontró localidad para villa alsina  (est. alsina) en baradero, buenos aires
Advertencia: No se encontró localidad para benito juarez  (e

#### **Tabla accesos_tecnologia_localidad**
- Se aplica una función para modificar los nombres de las provincias, localidades y partido a minúsculas.

In [41]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres a minúsculas
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `accesos_tecnologia_localidad.csv`
file_path = "datasets/dataset_procesados_proyecto/Internet_processed/accesos_tecnologia_localidad.csv"
df_accesos_tecnologia = pd.read_csv(file_path)

# Normalizar nombres de provincia, partido y localidad
df_accesos_tecnologia['Provincia'] = df_accesos_tecnologia['Provincia'].apply(normalizar_nombre)
df_accesos_tecnologia['Partido'] = df_accesos_tecnologia['Partido'].apply(normalizar_nombre)
df_accesos_tecnologia['Localidad'] = df_accesos_tecnologia['Localidad'].apply(normalizar_nombre)

# Transacción para la inserción de datos en `tecnologias_acceso`
try:
    for index, row in df_accesos_tecnologia.iterrows():
        # Obtener el id_provincia desde la tabla provincias
        cursor.execute("""
            SELECT id_provincia FROM provincias WHERE LOWER(nombre_provincia) = %s
        """, (row['Provincia'],))
        id_provincia = cursor.fetchone()

        if id_provincia is None:
            print(f"Advertencia: No se encontró provincia para {row['Provincia']}")
            continue  

        id_provincia = id_provincia[0]  # Obtener el id_provincia correspondiente

        # Obtener el id_localidad desde la tabla localidades
        cursor.execute("""
            SELECT id_localidad FROM localidades WHERE LOWER(localidad) = %s AND LOWER(partido) = %s AND id_provincia = %s
        """, (row['Localidad'], row['Partido'], id_provincia))
        id_localidad = cursor.fetchone()

        if id_localidad is None:
            print(f"Advertencia: No se encontró localidad para {row['Localidad']} en {row['Partido']}, {row['Provincia']}")
            continue  

        id_localidad = id_localidad[0]  # Obtener el id_localidad correspondiente

        # Insertar en la tabla tecnologias_acceso
        cursor.execute("""
            INSERT INTO tecnologias_acceso (id_localidad, adsl, cablemodem, dial_up, fibra_optica, otros, satelital, wimax, wireless, total_general)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            id_localidad,
            row['ADSL'],
            row['CABLEMODEM'],
            row['DIAL UP'],
            row['FIBRA OPTICA'],
            row['OTROS'],
            row['SATELITAL'],
            row['WIMAX'],
            row['WIRELESS'],
            row['Total general']
        ))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'tecnologias_acceso'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'tecnologias_acceso': {e}")

finally:
    cursor.close()
    connection.close()


Advertencia: No se encontró localidad para alfredo demarchi  (est. facundo quiroga) en 9 de julio, buenos aires
Advertencia: No se encontró localidad para manuel b. gonnet  (est. french) en 9 de julio, buenos aires
Advertencia: No se encontró localidad para villa general fournier  (est. 9 de julio sud) en 9 de julio, buenos aires
Advertencia: No se encontró localidad para adolfo gonzales chaves  (est. chaves) en adolfo gonzales chaves, buenos aires
Advertencia: No se encontró localidad para alberti   (est. andres vaccarezza) en alberti, buenos aires
Advertencia: No se encontró localidad para villa ortiz  (est. coronel mom) en alberti, buenos aires
Advertencia: No se encontró localidad para general daniel cerri  (est. general cerri) en bahia blanca, buenos aires
Advertencia: No se encontró localidad para villa alsina  (est. alsina) en baradero, buenos aires
Advertencia: No se encontró localidad para benito juarez  (est. juarez) en benito juarez, buenos aires
Advertencia: No se encontró 

#### **Tabla accesos_velocidad_provincia**

- **Problema de nombre "Capital Federal"**

Durante la inserción de datos en la tabla ``accesos_velocidad_provincia``, hubo un conflicto con la provincia "Capital Federal". El nombre de la provincia en el dataset original no coincidía con la forma en que se había registrado en la tabla provincias que se encuentra como ``"caba"``.

- **Solución aplicada:**

Para resolver este problema, se realizó una sustitución en el dataset original antes de la inserción en la base de datos. Se reemplazó todas las instancias de ``"Capital Federal"`` por ``"caba" ``para asegurar que los nombres coincidieran con los registros en la tabla provincias.

In [42]:
print(df_accesos_velocidad.columns)

Index(['Provincia', 'Partido', 'Localidad', 'Link Indec', 'Otros',
       '0,256 Mbps', '0,5 Mbps', '0,512 Mbps', '0,75 Mbps', '1 Mbps',
       ...
       '680 Mbps', '700 Mbps', '800 Mbps', '850 Mbps', '900 Mbps', '999 Mbps',
       '1000 Mbps', '1024 Mbps', '4000 Mbps', '10000 Mbps'],
      dtype='object', length=125)


In [43]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres de provincia
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `accesos_por_velocidad.csv`
file_path = "datasets/dataset_procesados_proyecto/Internet_processed/accesos_por_velocidad.csv"
df_accesos_velocidad = pd.read_csv(file_path)

# Se renombran columnas
df_accesos_velocidad.rename(columns={
    'HASTA 512 kbps': 'hasta_512_kbps',
    '+ 512 Kbps - 1 Mbps': 'entre_512_kbps_1_mbps',
    '+ 1 Mbps - 6 Mbps': 'entre_1_mbps_6_mbps',
    '+ 6 Mbps - 10 Mbps': 'entre_6_mbps_10_mbps',
    '+ 10 Mbps - 20 Mbps': 'entre_10_mbps_20_mbps',
    '+ 20 Mbps - 30 Mbps': 'entre_20_mbps_30_mbps',
    '+ 30 Mbps': 'mas_de_30_mbps',
    'OTROS': 'otros',
    'Total': 'total_accesos'
}, inplace=True)

# Reemplazar "Capital Federal" por "caba"
df_accesos_velocidad['Provincia'] = df_accesos_velocidad['Provincia'].replace({'Capital Federal': 'caba'})

# Normalizar los nombres de provincia en el DataFrame
df_accesos_velocidad['Provincia'] = df_accesos_velocidad['Provincia'].apply(normalizar_nombre)

# Obtener las provincias normalizadas desde la base de datos
cursor.execute("SELECT id_provincia, nombre_provincia FROM provincias")
provincias = cursor.fetchall()

# Diccionario con los nombres de provincia normalizados
provincias_normalizadas = {normalizar_nombre(prov[1]): prov[0] for prov in provincias}

# Transacción para la inserción de datos en `accesos_velocidad_provincia`
try:
    for index, row in df_accesos_velocidad.iterrows():
        # Obtener el id_provincia desde el diccionario
        id_provincia = provincias_normalizadas.get(normalizar_nombre(row['Provincia']))
        if id_provincia is None:
            print(f"Advertencia: No se encontró provincia para {row['Provincia']}")
            continue

        # Obtener el id_periodo
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla accesos_velocidad_provincia
        cursor.execute("""
            INSERT INTO accesos_velocidad_provincia (
                id_provincia, id_periodo, hasta_512_kbps, entre_512_kbps_1_mbps, entre_1_mbps_6_mbps,
                entre_6_mbps_10_mbps, entre_10_mbps_20_mbps, entre_20_mbps_30_mbps, mas_de_30_mbps,
                otros, total_accesos
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            id_provincia, id_periodo[0], row['hasta_512_kbps'], row['entre_512_kbps_1_mbps'],
            row['entre_1_mbps_6_mbps'], row['entre_6_mbps_10_mbps'], row['entre_10_mbps_20_mbps'],
            row['entre_20_mbps_30_mbps'], row['mas_de_30_mbps'], row['otros'], row['total_accesos']
        ))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'accesos_velocidad_provincia'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'accesos_velocidad_provincia': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'accesos_velocidad_provincia'


#### **Tabla penetracion_internet_poblacion**

- **Problema con el nombre "Capital Federal"**

Al igual que la ingesta anterior para la tabla ``accesos_velocidad_provincia`` donde surgía el conflicto con la *Capital Federal*, aquí se sigue la misma lógica para estandarizar el nombre y reemplazarlo por *CABA*.

In [44]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres de provincia
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `penetracion_internet_poblacion.csv`
file_path = "datasets/dataset_procesados_proyecto/Internet_processed/penetracion_internet_poblacion.csv"
df_penetracion_poblacion = pd.read_csv(file_path)

# Reemplazar "Capital Federal" por "caba"
df_penetracion_poblacion['Provincia'] = df_penetracion_poblacion['Provincia'].replace({'Capital Federal': 'caba'})

# Normalizar los nombres de provincia en el DataFrame
df_penetracion_poblacion['Provincia'] = df_penetracion_poblacion['Provincia'].apply(normalizar_nombre)

# Se obtiene las provincias normalizadas desde la base de datos
cursor.execute("SELECT id_provincia, nombre_provincia FROM provincias")
provincias = cursor.fetchall()

# Diccionario con los nombres de provincia normalizados
provincias_normalizadas = {normalizar_nombre(prov[1]): prov[0] for prov in provincias}

# Transacción para la inserción de datos en `penetracion_internet_poblacion`
try:
    for index, row in df_penetracion_poblacion.iterrows():
        # Obtener el id_provincia desde el diccionario
        id_provincia = provincias_normalizadas.get(normalizar_nombre(row['Provincia']))
        if id_provincia is None:
            print(f"Advertencia: No se encontró provincia para {row['Provincia']}")
            continue

        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla penetracion_internet_poblacion
        cursor.execute("""
            INSERT INTO penetracion_internet_poblacion (
                id_provincia, id_periodo, accesos_por_100_hab
            )
            VALUES (%s, %s, %s)
        """, (
            id_provincia, id_periodo[0], row['Accesos por cada 100 hab']
        ))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'penetracion_internet_poblacion'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'penetracion_internet_poblacion': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'penetracion_internet_poblacion'


#### **Tabla penetracion_internet_hogares**

- **Problema con el nombre "Capital Federal"**

Al igual que la ingesta anterior para la tabla ``penetracion_internet_poblacion`` donde surgía el conflicto con la *Capital Federal*, aquí se sigue la misma lógica para estandarizar el nombre y reemplazarlo por *CABA*.

In [45]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres de provincia
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `penetracion_internet_hogares.csv`
file_path = "datasets/dataset_procesados_proyecto/Internet_processed/penetracion_internet_hogares.csv"
df_penetracion_hogares = pd.read_csv(file_path)

# Reemplazar "Capital Federal" por "caba"
df_penetracion_hogares['Provincia'] = df_penetracion_hogares['Provincia'].replace({'Capital Federal': 'caba'})

# Normalizar los nombres de provincia en el DataFrame
df_penetracion_hogares['Provincia'] = df_penetracion_hogares['Provincia'].apply(normalizar_nombre)

# Se obtiene las provincias normalizadas desde la base de datos
cursor.execute("SELECT id_provincia, nombre_provincia FROM provincias")
provincias = cursor.fetchall()

# Diccionario con los nombres de provincia normalizados
provincias_normalizadas = {normalizar_nombre(prov[1]): prov[0] for prov in provincias}

# Transacción para la inserción de datos en `penetracion_internet_hogares`
try:
    for index, row in df_penetracion_hogares.iterrows():
        # Obtener el id_provincia desde el diccionario
        id_provincia = provincias_normalizadas.get(normalizar_nombre(row['Provincia']))
        if id_provincia is None:
            print(f"Advertencia: No se encontró provincia para {row['Provincia']}")
            continue

        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla penetracion_internet_hogares
        cursor.execute("""
            INSERT INTO penetracion_internet_hogares (
                id_provincia, id_periodo, accesos_por_100_hogares
            )
            VALUES (%s, %s, %s)
        """, (
            id_provincia, id_periodo[0], row['Accesos por cada 100 hogares']
        ))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'penetracion_internet_hogares'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'penetracion_internet_hogares': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'penetracion_internet_hogares'


#### **Tabla ingresos_servicios_internet**

In [46]:
import pandas as pd
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `ingresos_servicios_internet.csv`
file_path = "datasets/dataset_procesados_proyecto/Internet_processed/ingresos_servicios_internet.csv"
df_ingresos = pd.read_csv(file_path)

# Transacción para la inserción de datos en `ingresos_servicios_internet`
try:
    for index, row in df_ingresos.iterrows():
        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla ingresos_servicios_internet
        cursor.execute("""
            INSERT INTO ingresos_servicios_internet (
                id_periodo, ingresos_miles_pesos
            ) VALUES (%s, %s)
        """, (
            id_periodo[0], row['Ingresos (miles de pesos)']
        ))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'ingresos_servicios_internet'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'ingresos_servicios_internet': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'ingresos_servicios_internet'


#### **Tabla velocidad_media_provincia**
- **Problema con el nombre "Capital Federal"**

Al igual que la ingesta de las anteriores tablas donde surgía el conflicto con la *Capital Federal*, aquí se sigue la misma lógica para estandarizar el nombre y reemplazarlo por *CABA*.

In [47]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres de provincia
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset `velocidad_media_provincia.csv`
file_path = "datasets/dataset_procesados_proyecto/Internet_processed/velocidad_media_provincia.csv"
df_velocidad_media = pd.read_csv(file_path)

# Reemplazar "Capital Federal" por "caba"
df_velocidad_media['Provincia'] = df_velocidad_media['Provincia'].replace({'Capital Federal': 'caba'})

# Normalizar los nombres de provincia en el DataFrame
df_velocidad_media['Provincia'] = df_velocidad_media['Provincia'].apply(normalizar_nombre)

# Se obtiene las provincias normalizadas desde la base de datos
cursor.execute("SELECT id_provincia, nombre_provincia FROM provincias")
provincias = cursor.fetchall()

# Diccionario con los nombres de provincia normalizados
provincias_normalizadas = {normalizar_nombre(prov[1]): prov[0] for prov in provincias}

# Transacción para la inserción de datos en `velocidad_media_provincia`
try:
    for index, row in df_velocidad_media.iterrows():
        # Obtener el id_provincia desde el diccionario
        id_provincia = provincias_normalizadas.get(normalizar_nombre(row['Provincia']))
        if id_provincia is None:
            print(f"Advertencia: No se encontró provincia para {row['Provincia']}")
            continue

        # Obtener el id_periodo (aquí se asume que ya existe una tabla 'periodos' con los datos correctos)
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla velocidad_media_provincia
        cursor.execute("""
            INSERT INTO velocidad_media_provincia (
                id_periodo, id_provincia, mbps_media_bajada
            ) VALUES (%s, %s, %s)
        """, (
            id_periodo[0], id_provincia, row['Mbps (Media de bajada)']
        ))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'velocidad_media_provincia'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'velocidad_media_provincia': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'velocidad_media_provincia'


---
#### **Creación de tablas (DDL) - Dataset Telefonia_movil**

- **Esquema y Modelo Relacional**


Todas las tablas que incluyen ``Año``, ``Trimestre`` y ``Periodo`` hacen referencia a la tabla ``'periodos'``. Esto responde a la necesidad de evitar redundancia y mantener un modelo limpio, donde las entidades que se repiten en múltiples datasets están centralizadas. 

- **Decisión sobre el uso de llaves foráneas:**

Todas las relaciones entre tablas que comparten información común (como provincias y periodos) se gestionan a través de llaves foráneas. Esto asegura la integridad referencial en todo momento, garantizando que cualquier dato relacionado se almacene de manera consistente.

In [48]:
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

cursor = connection.cursor()

# Habilitamos autocommit
connection.autocommit = True


#Crear tabla 'sms_salientes'
cursor.execute("""
           
CREATE TABLE sms_salientes (
    id_sms SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo), -- Relación con la tabla periodos
    numero_sms_salientes BIGINT
);
               
""")


# Crear tabla 'llamadas_salientes'
cursor.execute("""

CREATE TABLE llamadas_salientes (
    id_llamada SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo),
    llamadas_pospago_miles DECIMAL(15, 2), -- Llamadas pospago en miles
    llamadas_prepago_miles DECIMAL(15, 2), -- Llamadas prepago en miles
    total_llamadas_salientes_miles DECIMAL(15, 2) -- Total de llamadas salientes en miles
);

""")


# Crear tabla 'minutos_salientes'
cursor.execute("""

CREATE TABLE minutos_salientes (
    id_minuto SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo),
    minutos_pospago_miles DECIMAL(15, 2), -- Minutos pospago en miles
    minutos_prepago_miles DECIMAL(15, 2), -- Minutos prepago en miles
    total_minutos_salientes_miles DECIMAL(15, 2) -- Total de minutos salientes en miles
);

""")


# Crear tabla 'ingresos_telefonia_movil'
cursor.execute("""
               
CREATE TABLE ingresos_telefonia_movil (
    id_ingreso_telefonia SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo),
    ingresos_miles_pesos DECIMAL(15, 2) -- Ingresos en miles de pesos
);
               
""")


# Crear tabla 'penetracion_telefonia_movil'
cursor.execute("""
               
CREATE TABLE penetracion_telefonia_movil (
    id_penetracion_telefonia SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo),
    accesos_por_100_hab DECIMAL(10, 2) -- Accesos por cada 100 habitantes
);

""")


# Crear tabla 'accesos_telefonia_movil'
cursor.execute("""
               
CREATE TABLE accesos_telefonia_movil (
    id_accesos_telefonia SERIAL PRIMARY KEY,
    id_periodo INT REFERENCES periodos(id_periodo),
    total_accesos_pospago DECIMAL(15, 2),
    total_accesos_prepago DECIMAL(15, 2),
    total_accesos_operativos INT
);
               
""")



print("Todas las tablas fueron creadas exitosamente.")


cursor.close()
connection.close()

Todas las tablas fueron creadas exitosamente.


#### **Ingesta de datos (DML) - Dataset Telefonia_movil**
- **Transacciones**

Se utilizan transacciones para garantizar que los datos se inserten de manera consistente. En caso de que ocurra un error durante la inserción, se hace un rollback para deshacer cualquier cambio.

#### **Tabla sms_salientes**

In [49]:
import pandas as pd
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset
file_path = "datasets/dataset_procesados_proyecto/Telefonia_processed/sms_salientes.csv"
df_sms_salientes = pd.read_csv(file_path)

# Transacción para la inserción de datos en 'sms_salientes'
try:
    for index, row in df_sms_salientes.iterrows():
        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla sms_salientes
        cursor.execute("""
            INSERT INTO sms_salientes (id_periodo, numero_sms_salientes)
            VALUES (%s, %s)
        """, (id_periodo[0], row['Número de SMS salientes']))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'sms_salientes'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'sms_salientes': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'sms_salientes'


#### **Tabla llamadas_salientes**

In [50]:
import pandas as pd
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset de llamadas salientes
file_path = "datasets/dataset_procesados_proyecto/Telefonia_processed/llamadas_salientes.csv"
df_llamadas_salientes = pd.read_csv(file_path)

# Transacción para la inserción de datos en 'llamadas_salientes'
try:
    for index, row in df_llamadas_salientes.iterrows():
        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla llamadas_salientes
        cursor.execute("""
            INSERT INTO llamadas_salientes (
                id_periodo, llamadas_pospago_miles, llamadas_prepago_miles, total_llamadas_salientes_miles
            )
            VALUES (%s, %s, %s, %s)
        """, (id_periodo[0], row['Llamadas pospago salientes (miles)'], row['Llamadas prepago salientes (miles)'], row['Total de llamadas salientes (miles)']))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'llamadas_salientes'")

except Exception as e:
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'llamadas_salientes': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'llamadas_salientes'


#### **Tabla minutos_salientes**

In [51]:
import pandas as pd
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset de minutos salientes
file_path = "datasets/dataset_procesados_proyecto/Telefonia_processed/minutos_salientes.csv"
df_minutos_salientes = pd.read_csv(file_path)

# Transacción para la inserción de datos en 'minutos_salientes'
try:
    for index, row in df_minutos_salientes.iterrows():
        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla minutos_salientes
        cursor.execute("""
            INSERT INTO minutos_salientes (
                id_periodo, minutos_pospago_miles, minutos_prepago_miles, total_minutos_salientes_miles
            )
            VALUES (%s, %s, %s, %s)
        """, (id_periodo[0], row['Minutos pospago salientes (miles)'], row['Minutos prepago salientes (miles)'], row['Total de minutos salientes (miles)']))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'minutos_salientes'")

except Exception as e:
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'minutos_salientes': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'minutos_salientes'


#### **Tabla ingresos_telefonia_movil**

In [52]:
import pandas as pd
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset
file_path = "datasets/dataset_procesados_proyecto/Telefonia_processed/ingresos_telefonia_movil.csv"
df_ingresos = pd.read_csv(file_path)

# Transacción para la inserción de datos en 'ingresos_telefonia_movil'
try:
    for index, row in df_ingresos.iterrows():
        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla ingresos_telefonia_movil
        cursor.execute("""
            INSERT INTO ingresos_telefonia_movil (id_periodo, ingresos_miles_pesos)
            VALUES (%s, %s)
        """, (id_periodo[0], row['Ingresos (miles de $)']))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'ingresos_telefonia_movil'")

except Exception as e:
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'ingresos_telefonia_movil': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'ingresos_telefonia_movil'


#### **Tabla penetracion_telefonia_movil**

In [53]:
import pandas as pd
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset de penetración telefonía móvil
file_path = "datasets/dataset_procesados_proyecto/Telefonia_processed/penetracion_telefonia_movil.csv"
df_penetracion = pd.read_csv(file_path)

# Transacción para la inserción de datos en 'penetracion_telefonia_movil'
try:
    for index, row in df_penetracion.iterrows():
        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla penetracion_telefonia_movil
        cursor.execute("""
            INSERT INTO penetracion_telefonia_movil (id_periodo, accesos_por_100_hab)
            VALUES (%s, %s)
        """, (id_periodo[0], row['Accesos por cada 100 hab']))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'penetracion_telefonia_movil'")

except Exception as e:
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'penetracion_telefonia_movil': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'penetracion_telefonia_movil'


#### **Tabla accesos_telefonia_movil**

In [54]:
import pandas as pd
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Cargar el dataset de accesos telefonía móvil
file_path = "datasets/dataset_procesados_proyecto/Telefonia_processed/accesos_telefonia_movil.csv"
df_accesos = pd.read_csv(file_path)

# Transacción para la inserción de datos en 'accesos_telefonia_movil'
try:
    for index, row in df_accesos.iterrows():
        # Obtener el id_periodo desde la tabla periodos
        cursor.execute("""
            SELECT id_periodo FROM periodos WHERE anio = %s AND trimestre = %s
        """, (row['Año'], row['Trimestre']))
        id_periodo = cursor.fetchone()

        if id_periodo is None:
            print(f"Advertencia: No se encontró periodo para Año {row['Año']}, Trimestre {row['Trimestre']}")
            continue

        # Insertar los datos en la tabla accesos_telefonia_movil
        cursor.execute("""
            INSERT INTO accesos_telefonia_movil (id_periodo, total_accesos_pospago, total_accesos_prepago, total_accesos_operativos)
            VALUES (%s, %s, %s, %s)
        """, (id_periodo[0], row['Total de accesos pospago'], row['Total de accesos prepago'], row['Total de accesos operativos']))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'accesos_telefonia_movil'")

except Exception as e:
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'accesos_telefonia_movil': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'accesos_telefonia_movil'


---
#### **Creación de tablas (DDL) - Dataset mapa_conectividad**

In [58]:
import psycopg2

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

cursor = connection.cursor()

# Habilitamos autocommit
connection.autocommit = True


#Crear tabla 'sms_salientes'
cursor.execute("""
           
CREATE TABLE mapa_conectividad (
    id_conectividad SERIAL PRIMARY KEY,
    id_localidad INT REFERENCES localidades(id_localidad),
    poblacion INT,
    adsl BOOLEAN,
    cablemodem BOOLEAN,
    dial_up BOOLEAN,
    fibra_optica BOOLEAN,
    satelital BOOLEAN,
    wireless BOOLEAN,
    telefonia_fija BOOLEAN,
    cobertura_3g BOOLEAN,
    cobertura_4g BOOLEAN,
    link_indec BIGINT,
    latitud DECIMAL(10, 7),
    longitud DECIMAL(10, 7)
);

               
""")

#### **Ingesta de datos (DML) - Dataset mapa_conectividad**

- **Transacciones**

Se utilizan transacciones para garantizar que los datos se inserten de manera consistente. En caso de que ocurra un error durante la inserción, se hace un rollback para deshacer cualquier cambio.

- **Normalización de valores booleanos** 

Se utiliza la función convertir_booleano para transformar "SI" y "--" en valores True y False que puedan ser aceptados por la base de datos.

- **Normalización de nombre de provincias** 

Se aplica una función para modificar los nombres de las provincias, localidades y partido a minúsculas.

- **Ingesta de datos** 

El código realiza una inserción de datos para cada fila, buscando el id_localidad correspondiente en la tabla localidades y utilizando transacciones para garantizar la consistencia de los datos.

#### **Tabla mapa_conectividad**

In [59]:
import pandas as pd
import psycopg2
import unicodedata

# Función para normalizar nombres a minúsculas
def normalizar_nombre(nombre):
    # Eliminar tildes, caracteres especiales y convertir a minúsculas
    return ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()

# Función para convertir "SI" y "--" en valores booleanos
def convertir_booleano(valor):
    if valor == "SI":
        return True
    elif valor == "--":
        return False
    return None

# Cargar el dataset 'mapa_conectividad.csv'
file_path = "datasets/dataset_procesados_proyecto/Mapa_conectividad_processed/mapa_conectividad.csv"
df_mapa_conectividad = pd.read_csv(file_path)

# Normalizar nombres de provincia, partido y localidad
df_mapa_conectividad['Provincia'] = df_mapa_conectividad['Provincia'].apply(normalizar_nombre)
df_mapa_conectividad['Partido'] = df_mapa_conectividad['Partido'].apply(normalizar_nombre)
df_mapa_conectividad['Localidad'] = df_mapa_conectividad['Localidad'].apply(normalizar_nombre)

# Conexión a la base de datos
connection = psycopg2.connect(
    user="TELECOM_USER",
    password="TELECOM_PASSWORD",
    host="localhost",
    port="5432",
    database="telecomunicaciones"
)

# Habilitar autocommit para manejar transacciones manualmente
connection.autocommit = False

# Cursor
cursor = connection.cursor()

# Transacción para la inserción de datos en 'mapa_conectividad'
try:
    for index, row in df_mapa_conectividad.iterrows():
        # Obtener el id_localidad desde la tabla localidades 
        cursor.execute("""
            SELECT id_localidad FROM localidades WHERE LOWER(localidad) = %s AND LOWER(partido) = %s
        """, (row['Localidad'], row['Partido']))
        id_localidad = cursor.fetchone()
        
        if id_localidad is None:
            print(f"Advertencia: No se encontró localidad para {row['Localidad']} en {row['Partido']}, {row['Provincia']}")
            continue
        
        # Convertir "SI" y "--" a valores booleanos
        adsl = convertir_booleano(row['ADSL'])
        cablemodem = convertir_booleano(row['Cablemódem'])
        dial_up = convertir_booleano(row['Dial Up'])
        fibra_optica = convertir_booleano(row['Fibra óptica'])
        satelital = convertir_booleano(row['Satelital'])
        wireless = convertir_booleano(row['Wireless'])
        telefonia_fija = convertir_booleano(row['Telefonía Fija'])
        cobertura_3g = convertir_booleano(row['3G'])
        cobertura_4g = convertir_booleano(row['4G'])

        # Insertar los datos en la tabla 'mapa_conectividad'
        cursor.execute("""
            INSERT INTO mapa_conectividad (
                id_localidad, poblacion, adsl, cablemodem, dial_up, fibra_optica, satelital, 
                wireless, telefonia_fija, cobertura_3g, cobertura_4g, link_indec, latitud, longitud
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            id_localidad[0], row['Población'], adsl, cablemodem, dial_up, fibra_optica, satelital,
            wireless, telefonia_fija, cobertura_3g, cobertura_4g, row['Link'], row['Latitud'], row['Longitud']
        ))

    # Confirmar la transacción
    connection.commit()
    print("Datos insertados correctamente en la tabla 'mapa_conectividad'")

except Exception as e:
    # En caso de error, hacer rollback para deshacer cualquier cambio
    connection.rollback()
    print(f"Error durante la inserción en la tabla 'mapa_conectividad': {e}")

finally:
    cursor.close()
    connection.close()


Datos insertados correctamente en la tabla 'mapa_conectividad'
